# Create the Database
This notebook creates the database used by the app to store data. This step is required before running the app.

### Set up Database Schema (DuckDB)
Tables: <br>
    - Volumes - store controller volumes <br>
    - Occupancies - store controller occupancies<br>
    - Logs - store event/error logs<br>
    - Signals - list of traffic signals in corridor<br>
    Note that Signals contains a column "detectors" which is a list of detector numbers from that signal that will be used. Storing detector numbers like this is not a best practice, but works for now. <br>
    - Settings - track user settings<br>

In [1]:
import duckdb

# Set interval period (in minutes, as a string)
interval = '5'

# Create new database with DuckDB
con = duckdb.connect(database='database.db', read_only=False)

# Create tables. Timestamps are set to the current time interval when the data is inserted.

# Volumes table
con.execute(f"""
            CREATE OR REPLACE TABLE Volumes (
                signal VARCHAR, 
                detector UTINYINT, 
                timestamp DATETIME DEFAULT time_bucket(INTERVAL '{interval} minutes', current_timestamp::TIMESTAMP),
                volume USMALLINT
            )
            """)
# Occupancies table
con.execute(f"""
            CREATE OR REPLACE TABLE Occupancies (
                signal VARCHAR, 
                detector UTINYINT, 
                timestamp DATETIME DEFAULT time_bucket(INTERVAL '{interval} minutes', current_timestamp::TIMESTAMP),
                occupancy FLOAT
            )
            """)
# Logs table
con.execute("""CREATE OR REPLACE TABLE Logs (
                timestamp DATETIME DEFAULT current_timestamp::TIMESTAMP,
                event VARCHAR,
                message VARCHAR
            )
            """)
# Signals table
con.execute("""CREATE OR REPLACE TABLE Signals (
                signal VARCHAR PRIMARY KEY, 
                ip VARCHAR, 
                detectors VARCHAR, 
                UNIQUE(ip)
            )
            """)
# Settings table
con.execute("""CREATE OR REPLACE TABLE Settings (
                setting VARCHAR PRIMARY KEY,
                value VARCHAR
            )
            """)

Populate tables with default values and traffic signals

In [2]:
# Insert default settings
con.execute(f"INSERT INTO Settings VALUES ('interval', '{interval}')") # Minutes between each update, should match controller detector zone interval
con.execute("INSERT INTO Settings VALUES ('mode', 'Traffic Predictive')") # default mode of operation
con.execute("INSERT INTO Settings VALUES ('debug', 'True')") # default debug mode
con.execute("INSERT INTO Settings VALUES ('run_app', 'True')") # default run app mode

con.query("SELECT * FROM Settings")

┌──────────┬────────────────────┐
│ setting  │       value        │
│ varchar  │      varchar       │
├──────────┼────────────────────┤
│ interval │ 5                  │
│ mode     │ Traffic Predictive │
│ debug    │ True               │
│ run_app  │ True               │
└──────────┴────────────────────┘

In [ ]:
# Insert traffic signals (IP addresses x'ed out because they are not public info)
con.execute("INSERT INTO Signals VALUES ('OR34_15th', 'xxx.xxx.xxx.xx', '2,16')") # 2,16 are the detector numbers
con.execute("INSERT INTO Signals VALUES ('OR34_53rd', 'xxx.xxx.xxx.xx', '2,16')") # 2,16 are the detector numbers

In [8]:
# Don't forget to close the connection or the app won't be able to access the database
con.close()